In [2]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key from the .env file
API_KEY = os.getenv('ticketmaster_api_key')
BASE_URL = 'https://app.ticketmaster.com/discovery/v2/events.json'

def fetch_events(city, state_code, size=10):
    params = {
        'apikey': API_KEY,
        'city': city,
        'stateCode': state_code,
        'size': size,
        'sort': 'date,asc'  # Sort events by date in ascending order
    }
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

def parse_events(data):
    events = data.get('_embedded', {}).get('events', [])
    event_list = []
    for event in events:
        event_info = {
            'Name': event.get('name'),
            'Date': event.get('dates', {}).get('start', {}).get('localDate'),
            'Time': event.get('dates', {}).get('start', {}).get('localTime'),
            'Venue': event.get('_embedded', {}).get('venues', [{}])[0].get('name'),
            'City': event.get('_embedded', {}).get('venues', [{}])[0].get('city', {}).get('name'),
            'State': event.get('_embedded', {}).get('venues', [{}])[0].get('state', {}).get('stateCode'),
            'URL': event.get('url')
        }
        event_list.append(event_info)
    return event_list

def main():
    city = 'San Diego'
    state_code = 'CA'
    data = fetch_events(city, state_code)
    if data:
        events = parse_events(data)
        df = pd.DataFrame(events)
        print(df)
    else:
        print("No data retrieved.")

if __name__ == "__main__":
    main()


                           Name        Date      Time  \
0           24 Post Season Test  2024-12-28  20:00:00   
1    2024 Season Ticket Deposit  2024-12-30  08:00:00   
2        2024 Wild Horses Suite  2024-12-30  18:00:00   
3               2024 NYE Proper  2024-12-31  11:00:00   
4        2024 Black Friday Deal  2024-12-31  17:00:00   
5                  Concert Test  2024-12-31  17:00:00   
6          2025 Alaska Airlines  2024-12-31  17:00:00   
7      Snapdragon Stadium Tours  2025-01-02      None   
8  Test Event - Do Not Purchase  2025-01-02  16:30:00   
9                    Black Flag  2025-01-02  19:00:00   

                           Venue       City State  \
0                     Petco Park  San Diego    CA   
1             Snapdragon Stadium  San Diego    CA   
2                     Petco Park  San Diego    CA   
3                     Petco Park  San Diego    CA   
4             Snapdragon Stadium  San Diego    CA   
5                     Petco Park  San Diego    CA   
6